In [1]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

In [2]:
import glob
import re
import yaml
import numpy as np
from tensorflow.keras import layers, activations, models, preprocessing
from tensorflow.keras import preprocessing, utils
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [3]:
ls

 Datentr„ger in Laufwerk F: ist HAW
 Volumeseriennummer: 3088-94EC

 Verzeichnis von F:\Semester 5\LA-Projekt\Final\ChatBotSeq2SeqBigData

12.06.2020  22:33    <DIR>          .
12.06.2020  22:33    <DIR>          ..
12.06.2020  18:51    <DIR>          .ipynb_checkpoints
12.06.2020  22:33            12.383 ChatBotBigData.ipynb
12.06.2020  19:02    <DIR>          dataset
               1 Datei(en),         12.383 Bytes
               4 Verzeichnis(se), 167.281.250.304 Bytes frei


In [4]:
batch_size = 64  # Batch size for training.
epochs = 1  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
embedding_dim = 100

In [5]:
dir_path='F:\Semester 5\LA-Projekt\Final\ChatBotSeq2SeqBigData\RedditDataset'
os.chdir(r'F:\Semester 5\LA-Projekt\Final\ChatBotSeq2SeqBigData\RedditDataset')
file_list = glob.glob('*.txt')
print(file_list)

['reddit_cleaned_part.txt']


In [6]:
questions = list()
answers = list()
#print(os.sep)
for file_path in file_list:
    stream = open(dir_path + os.sep + file_path, 'rb')
    f = open(file_path, "r")
    #print("hi")
    for line in f:
        if line[0] =="Q":
            line=re.sub('Q: ', '', line)
            #line=re.sub('_.*_', ' ',line)
            questions.append(line)
        elif line[0]=="A":
            line=re.sub('A: ', '', line)
            line=re.sub('_func.*',' ', line)
            answers.append(line)
t_answers=list()
for an in range(len(answers)):
    t_answers.append('<START> ' + answers[an] + ' <END>')

In [7]:
def tokenize(sentences):
    sentences_clear = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('can\'t', 'can not', sentence)
        sentence = re.sub('n\'t', ' not', sentence)
        sentence = re.sub('\'ve', ' have', sentence)
        sentence = re.sub('\'ll', ' will', sentence)
        sentence = re.sub('\'s', ' is', sentence)
        sentence = re.sub('\'m', ' am', sentence)
        sentence = re.sub('\'re', ' are', sentence)
        sentence = re.sub('\'d', ' would', sentence)
        sentence = re.sub('\n', '', sentence)
        #sentence = re.sub('_func.*', ' ', sentence)
        sentences_clear.append(sentence)
    return sentences_clear
#print(answers[2])
answersTok = tokenize(t_answers)
questionsTok = tokenize(questions)
print(len(answers))
#for i in range(100):
#    print(answers[i])
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
word_index = tokenizer.word_index
VOCAB_SIZE = len(word_index) + 1
print('Found %s unique tokens.' % VOCAB_SIZE)

116915
Found 54803 unique tokens.


In [8]:
print(questions[0:5])

['Is it supposed to be like enhanced if they convert enhanced orbs to poison ?\n', "My PSU does n't have that switch .\n", "Well that 's not an accurate statement . Scalise now carries plenty of baggage , just like Mr. Duke .\n", "Im moving from St Pete to Denver in two weeks . F ' Pam Bondi and Rick Scott !\n", "This is n't just signed . Groening actually drew Akbar and Jeff on it . You own an original Matt Groening cartoon .\n"]


In [9]:
indices = np.arange(len(questions))
np.random.shuffle(indices)
answers = list()
questions = list()
for i in indices:
    answers.append(answersTok[i])
    questions.append(questionsTok[i])

In [10]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
print(maxlen_questions)
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
print(maxlen_answers)

32
34


In [11]:
encoder_inputs = tf.keras.layers.Input(shape=(None,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(latent_dim, dropout=0.2,
                                                         recurrent_dropout=0.2,
                                                         return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(latent_dim, dropout=0.2, recurrent_dropout=0.2, return_state=True,
                                    return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE, activation=tf.keras.activations.softmax)
output = decoder_dense(decoder_outputs)

In [12]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    5480300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    5480300     input_2[0][0]                    
______________________________________________________________________________________________

In [13]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['acc'])

In [26]:
dataToTake = 1500

In [24]:
startIndex =  0
toIndex = dataToTake
ansBatchToTake = tokenized_answers[startIndex:toIndex]
queBatchToTake = tokenized_questions[startIndex:toIndex]
print(len(ansBatchToTake))

2000


In [25]:
# encoder_input_data
encoder_input_data = preprocessing.sequence.pad_sequences(queBatchToTake, maxlen=maxlen_questions, padding='post')
print(encoder_input_data.shape)

# decoder_input_data
decoder_input_data = preprocessing.sequence.pad_sequences(ansBatchToTake, maxlen=maxlen_answers, padding='post')
print(decoder_input_data.shape)

# decoder_output_data
for i in range(len(ansBatchToTake)):
    ansBatchToTake[i] = ansBatchToTake[i][1:]
decoder_output_data = preprocessing.sequence.pad_sequences(ansBatchToTake, maxlen=maxlen_answers, padding='post')
decoder_output_data = utils.to_categorical(decoder_output_data, VOCAB_SIZE)
print(decoder_output_data.shape)

(2000, 32)
(2000, 34)


MemoryError: Unable to allocate array with shape (68000, 54803) and data type float32

In [ ]:
history = model.fit([encoder_input_data, decoder_input_data],
                    decoder_output_data,
                    steps_per_epoch = int(len(queBatchToTake) // batch_size),
                    epochs=epochs)

In [ ]:
model.save('Test.h5')